In [1]:
import numpy as np
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from src.score import *
import re

import generate_data as gd

In [2]:
class PeriodicPadding2D(tf.keras.layers.Layer):
    def __init__(self, pad_width, **kwargs):
        super().__init__(**kwargs)
        self.pad_width = pad_width

    def call(self, inputs, **kwargs):
        if self.pad_width == 0:
            return inputs
        inputs_padded = tf.concat(
            [inputs[:, :, -self.pad_width:, :], inputs, inputs[:, :, :self.pad_width, :]], axis=2)
        # Zero padding in the lat direction
        inputs_padded = tf.pad(inputs_padded, [[0, 0], [self.pad_width, self.pad_width], [0, 0], [0, 0]])
        return inputs_padded

    def get_config(self):
        config = super().get_config()
        config.update({'pad_width': self.pad_width})
        return config


class PeriodicConv2D(tf.keras.layers.Layer):
    def __init__(self, filters,
                 kernel_size,
                 conv_kwargs={},
                 **kwargs, ):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.conv_kwargs = conv_kwargs
        if type(kernel_size) is not int:
            assert kernel_size[0] == kernel_size[1], 'PeriodicConv2D only works for square kernels'
            kernel_size = kernel_size[0]
        pad_width = (kernel_size - 1) // 2
        self.padding = PeriodicPadding2D(pad_width)
        self.conv = Conv2D(
            filters, kernel_size, padding='valid', **conv_kwargs
        )

    def call(self, inputs):
        return self.conv(self.padding(inputs))

    def get_config(self):
        config = super().get_config()
        config.update({'filters': self.filters, 'kernel_size': self.kernel_size, 'conv_kwargs': self.conv_kwargs})
        return config

In [3]:
def build_cnn(filters, kernels, input_shape, dr=0):
    """Fully convolutional network"""
    x = input = Input(shape=input_shape)
    for f, k in zip(filters[:-1], kernels[:-1]):
        x = PeriodicConv2D(f, k)(x)
        x = LeakyReLU()(x)
    output = PeriodicConv2D(filters[-1], kernels[-1])(x)
    return keras.models.Model(input, output)

def create_predictions(model, dg):
    """Create non-iterative predictions"""
    preds_un = xr.DataArray(
        model.predict(dg),
        dims=['time', 'lat', 'lon', 'level'],
        coords={'time': dg.valid_time, 'lat': dg.data.lat, 'lon': dg.data.lon,
                'level': dg.data.isel(level=dg.output_idxs).level,
                'level_names': dg.data.isel(level=dg.output_idxs).level_names
               },
    )
    # Unnormalize
    preds = (preds_un * dg.std.isel(level=dg.output_idxs).values +
             dg.mean.isel(level=dg.output_idxs).values)
    unique_vars = list(set([l.split('_')[0] for l in preds.level_names.values])); unique_vars

    das = []
    for v in unique_vars:
        idxs = [i for i, vv in enumerate(preds.level_names.values) if vv.split('_')[0] in v]
        #print(v, idxs)
        da = preds.isel(level=idxs).squeeze().drop('level_names')
        if not 'level' in da.dims: da.drop('level')
        das.append({v: da})
    return preds_un, xr.merge(das, compat = 'override').drop('level')

In [4]:
def predictions(var_name, valid):
    dg_train, dg_valid, dg_test = gd.create_data(var_name)

    if var_name == 'specific_humidity':
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 4))
    elif var_name == 'pot_vort':
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 4))
    elif var_name == 'const':
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 5))
    elif var_name == 'orig':
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 2))
    else:
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 3))

    cnn.compile(keras.optimizers.Adam(1e-4), 'mse')

    cnn.load_weights('/rds/general/user/mc4117/home/WeatherBench/saved_models/72_gpu_annual' + str(var_name) + '.h5')

    number_of_forecasts = 1

    pred_ensemble=np.ndarray(shape=(2, 17448, 32, 64, number_of_forecasts),dtype=np.float32)
    print(pred_ensemble.shape)
    forecast_counter=np.zeros(number_of_forecasts,dtype=int)

    for j in range(number_of_forecasts):
        print(j)
        preds, output = create_predictions(cnn, dg_test)
        pred2 = np.asarray(output.to_array(), dtype=np.float32).squeeze()
        pred_ensemble[:,:,:,:,j]=pred2
        forecast_counter[j]=j+1
        filename2 = '/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annual' + str(var_name)
        np.save(filename2 + '.npy', pred_ensemble)
        


    return preds, compute_weighted_rmse(output, valid).compute()

In [5]:
DATADIR = '/rds/general/user/mc4117/home/WeatherBench/data/'

z500_valid = load_test_data(f'{DATADIR}geopotential_500', 'z')
t850_valid = load_test_data(f'{DATADIR}temperature_850', 't')
valid = xr.merge([z500_valid, t850_valid])


In [6]:
preds_orig, rmse = predictions('orig', valid)
rmse

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.227
    z        float64 708.7

In [7]:
preds_sh, rmse_sh = predictions('specific_humidity', valid)
rmse_sh

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.236
    z        float64 718.3

In [8]:
preds_pv, rmse_pv = predictions('pot_vort', valid)
rmse_pv

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.225
    z        float64 714.3

In [9]:
preds_const, rmse_const = predictions('const', valid)
rmse_const

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.173
    z        float64 713.4

In [10]:
preds_2mt, rmse_2mt = predictions('2m temp', valid)
rmse_2mt

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.188
    z        float64 708.3

In [11]:
preds_sr, rmse_sr = predictions('solar rad', valid)
rmse_sr

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    t        float64 3.222
    z        float64 709.0

In [12]:
#output_orig = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualorig.npy')
#output_sh = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualspecific_humidity.npy')
#output_pv = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualpot_vort.npy')
#output_const = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualconst.npy')
#output_2mt = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annual2m temp.npy')
#output_sr = np.load('/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annualsolar rad.npy')

In [13]:
#output_mean = sum([output_sh + output_pv + output_const + output_2mt + output_sr])/5

#dg_train, dg_valid, dg_test = gd.create_data('orig')

#output_mean_ds = xr.Dataset({
#    'z': xr.DataArray(
#        output_mean[0, ..., 0],
#        dims=['time', 'lat', 'lon'],
#        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
#                },
#    ),
#    't': xr.DataArray(
#        output_mean[1, ..., 0],
#        dims=['time', 'lat', 'lon'],
#        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
#                },
#    )
#})


In [14]:
#compute_weighted_rmse(output_mean_ds, valid).compute()

In [15]:
stack_out = np.stack([preds_orig, preds_sh, preds_pv, preds_const, 
                      preds_2mt,  preds_sr])  

In [18]:
dg_train, dg_valid, dg_test = gd.create_data('orig')
X1, y1 = dg_test[0]

for i in range(1, len(dg_test)):
    X2, y2 = dg_test[i]
    y1 = np.concatenate((y1, y2))

Loading data into RAM
Loading data into RAM
Loading data into RAM


In [24]:
stack_out.shape

(6, 17448, 32, 64, 2)

In [ ]:
stack_test_list = []

for i in range(6):
    stack_test_list.append(stack_out[i][..., 0])

In [ ]:
print(stack_test_list[0].shape)
print(y1[..., 0].shape)

In [226]:
from tensorflow.keras.layers import concatenate

def my_init(shape, dtype=None):
    print(shape)
    return tf.ones(shape, dtype=dtype)/6

def build_stack_model(input_shape, stack_list):
    # concatenate merge output from each model
    input_list = [Input(shape=input_shape) for i in range(len(stack_list))]
    merge = concatenate(input_list)
    hidden = Dense(6, activation='relu', kernel_initializer = my_init)(merge)
    hidden = Dense(64, activation='relu')(hidden)
    hidden = Dense(64, activation='relu')(hidden)
    hidden = Dense(64, activation='relu')(hidden)
    output = Dense(64, activation='relu')(hidden)
    return keras.models.Model(input_list, output)

In [227]:
ensemble_model = build_stack_model((32, 64), stack_test_list)

ensemble_model.compile(keras.optimizers.Adam(1e-4), 'mse')

def scheduler(epoch, lr):
    if epoch< 5:
        return lr #/(2*epoch+1)
    elif epoch < 10:
        return lr/10
    elif epoch < 15:
        return lr/20
    else:
        return lr/100

callback = tf.keras.callbacks.LearningRateScheduler(scheduler)   

ensemble_model.fit(x = stack_test_list, y = y1[...,0], epochs = 20, validation_split = 0.2) #, shuffle = True) #, callbacks = [callback])

[384, 6]
Train on 13958 samples, validate on 3490 samples
Epoch 1/20
13958/13958 [==============================] - 10s 729us/sample - loss: 1.8257 - val_loss: 0.7276
Epoch 2/20
13958/13958 [==============================] - 8s 598us/sample - loss: 0.6824 - val_loss: 0.7221
Epoch 3/20
13958/13958 [==============================] - 9s 613us/sample - loss: 0.6782 - val_loss: 0.7162
Epoch 4/20
13958/13958 [==============================] - 9s 634us/sample - loss: 0.6754 - val_loss: 0.7161
Epoch 5/20
13958/13958 [==============================] - 10s 745us/sample - loss: 0.6753 - val_loss: 0.7160
Epoch 6/20
13958/13958 [==============================] - 8s 608us/sample - loss: 0.6723 - val_loss: 0.7101
Epoch 7/20
13958/13958 [==============================] - 9s 613us/sample - loss: 0.6580 - val_loss: 0.6925
Epoch 8/20
13958/13958 [==============================] - 9s 658us/sample - loss: 0.6512 - val_loss: 0.6922
Epoch 9/20
13958/13958 [==============================] - 9s 659us/sample - 

In [228]:
fc = ensemble_model.predict(stack_test_list)
preds_un = xr.DataArray(
        fc,
        dims=['time', 'lat', 'lon'],
        coords={'time': dg_test.valid_time, 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
               },
    )
    # Unnormalize
preds = (preds_un * dg_test.std.isel(level=0).values +
             dg_test.mean.isel(level=0).values)

compute_weighted_rmse(preds, real_ds).compute()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 1.976e+03

In [198]:
    stop
    preds_un = xr.DataArray(
        fc,
        dims=['time', 'lat', 'lon', 'level'],
        coords={'time': dg_test.valid_time, 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                'level': dg_test.data.isel(level=dg_test.output_idxs).level,
                'level_names': dg_test.data.isel(level=dg_test.output_idxs).level_names
               },
    )
    # Unnormalize
    preds = (preds_un * dg_test.std.isel(level=dg_test.output_idxs).values +
             dg_test.mean.isel(level=dg_test.output_idxs).values)
    unique_vars = list(set([l.split('_')[0] for l in preds.level_names.values])); unique_vars

    das = []
    for v in unique_vars:
        idxs = [i for i, vv in enumerate(preds.level_names.values) if vv.split('_')[0] in v]
        #print(v, idxs)
        da = preds.isel(level=idxs).squeeze().drop('level_names')
        if not 'level' in da.dims: da.drop('level')
        das.append({v: da})
    fc_unnorm = xr.merge(das, compat = 'override').drop('level')

ValueError: different number of dimensions on data and dims: 3 vs 4

In [181]:
stop
real_unnorm =y1* dg_test.std.isel(level=[0,1]).values+dg_test.mean.isel(level=[0,1]).values

real_ds = xr.Dataset({
    'z': xr.DataArray(
        real_unnorm[..., 0],
        dims=['time', 'lat', 'lon'],
        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                },
    ),
    't': xr.DataArray(
        real_unnorm[..., 1],
        dims=['time', 'lat', 'lon'],
        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                },
    )
})

In [203]:
real_unnorm =y1* dg_test.std.isel(level=[0]).values+dg_test.mean.isel(level=[0]).values

real_ds = xr.Dataset({
    'z': xr.DataArray(
        real_unnorm[..., 0],
        dims=['time', 'lat', 'lon'],
        coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                },
    ),
})

## Linear Regression

In [229]:
print(stack_test_list[0].shape)
print(y1[..., 0].shape)

(17448, 32, 64)
(17448, 32, 64)


In [230]:
_, nlat, nlon = stack_test_list[0].shape; nlat, nlon

(32, 64)

In [234]:
stack_reshape = [stack_test_list[i].reshape(-1, nlat*nlon) for i in range(len(stack_test_list))]

y_reshape = y1[...,0].reshape(-1, nlat*nlon)   

In [246]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(stack_total, y_reshape)

LinearRegression()

In [245]:
stack_total = stack_reshape[0] + stack_reshape[1] + stack_reshape[2] + stack_reshape[3] + stack_reshape[4] + stack_reshape[5]

In [247]:
preds = lr.predict(stack_total).reshape((-1, nlat, nlon))

In [250]:
fc = xr.DataArray(
            preds * dg_test.std.isel(level=[0]).values+dg_test.mean.isel(level=[0]).values,
            dims=['time', 'lat', 'lon'],
            coords={'time':dg_test.data.time[72:], 'lat': dg_test.data.lat, 'lon': dg_test.data.lon,
                },
    )

In [251]:
compute_weighted_rmse(fc, real_ds).compute()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 184.6

In [255]:
# Testing linear regression

def testing_predictions(var_name):
    dg_train, dg_valid, dg_test = gd.create_data(var_name)

    if var_name == 'specific_humidity':
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 4))
    elif var_name == 'pot_vort':
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 4))
    elif var_name == 'const':
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 5))
    elif var_name == 'orig':
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 2))
    else:
        cnn = build_cnn([64, 64, 64, 64, 2], [5, 5, 5, 5, 5], (32, 64, 3))

    cnn.compile(keras.optimizers.Adam(1e-4), 'mse')

    cnn.load_weights('/rds/general/user/mc4117/home/WeatherBench/saved_models/72_gpu_annual' + str(var_name) + '.h5')

    preds, output = create_predictions(cnn, dg_valid)     

    return preds, output

In [256]:
preds_test, output_test = testing_predictions('orig')
preds_test_sh, output_test_sh = testing_predictions('specific_humidity')
preds_test_pv, output_test_pv = testing_predictions('pot_vort')
preds_test_2mt, output_test_2mt = testing_predictions('2m temp')
preds_test_const, output_test_const = testing_predictions('const')
preds_test_sr, output_test_sr = testing_predictions('solar rad')


Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM
Loading data into RAM


In [260]:
stack_out_test = np.stack([preds_test, preds_test_sh, preds_test_pv, preds_test_const, 
                      preds_test_2mt,  preds_test_sr])  

In [258]:
dg_train, dg_valid, dg_test = gd.create_data('orig')
X1, y1 = dg_valid[0]

for i in range(1, len(dg_valid)):
    X2, y2 = dg_valid[i]
    y1 = np.concatenate((y1, y2))

Loading data into RAM
Loading data into RAM
Loading data into RAM


In [261]:
stack_test_list2 = []

for i in range(6):
    stack_test_list2.append(stack_out_test[i][..., 0])

In [262]:
stack_reshape2 = [stack_test_list2[i].reshape(-1, nlat*nlon) for i in range(len(stack_test_list2))]

stack_total2 = stack_reshape2[0] + stack_reshape2[1] + stack_reshape2[2] + stack_reshape2[3] + stack_reshape2[4] + stack_reshape2[5]

In [263]:
preds_valid = lr.predict(stack_total2).reshape((-1, nlat, nlon))

In [265]:
fc_valid = xr.DataArray(
            preds_valid * dg_valid.std.isel(level=[0]).values+dg_valid.mean.isel(level=[0]).values,
            dims=['time', 'lat', 'lon'],
            coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                },
    )

In [266]:
real_unnorm_valid =y1* dg_valid.std.isel(level=[0]).values+dg_valid.mean.isel(level=[0]).values

real_ds_valid = xr.Dataset({
    'z': xr.DataArray(
        real_unnorm_valid[..., 0],
        dims=['time', 'lat', 'lon'],
        coords={'time':dg_valid.data.time[72:], 'lat': dg_valid.data.lat, 'lon': dg_valid.data.lon,
                },
    ),
})

In [267]:
compute_weighted_rmse(fc_valid, real_ds_valid).compute()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 1.032e+03